In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import cv2,os
data_path=r"/content/drive/My Drive/Colab Notebooks/dataset"
categories=os.listdir(data_path)
labels=[i for i in range(len(categories))]

label_dict=dict(zip(categories,labels))

print(label_dict)
print(categories)
print(labels)

FileNotFoundError: ignored

In [ ]:
img_size=100
data=[]
target=[]


for category in categories:
    folder_path=os.path.join(data_path,category)
    img_names=os.listdir(folder_path)
        
    for img_name in img_names:
        img_path=os.path.join(folder_path,img_name)
        img=cv2.imread(img_path)

        try:
            gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)           
           
            resized=cv2.resize(gray,(img_size,img_size))
            
            data.append(resized)
            target.append(label_dict[category])
            

        except Exception as e:
            print('Exception:',e)
           

In [ ]:
import numpy as np
data=np.array(data)/255.0
data=np.reshape(data,(data.shape[0],img_size,img_size,1))
target=np.array(target)
from keras.utils import np_utils
new_target=np_utils.to_categorical(target)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(data,new_target,test_size=0.1)

In [ ]:
X_train.shape,X_test.shape,Y_train.shape,Y_test.shape

In [ ]:
np.save(r"C:\Users\LEN\Downloads\New_data",data)
np.save(r"C:\Users\LEN\Downloads\New_target",new_target)

In [ ]:
import numpy as np
data=np.load(r"C:\Users\LEN\Downloads\New_data.npy")
new_target=np.load(r"C:\Users\LEN\Downloads\New_target.npy")

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.layers import Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint

model=Sequential()

model.add(Conv2D(200,(3,3),input_shape=data.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#The first CNN layer followed by Relu and MaxPooling layers

model.add(Conv2D(100,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#The second convolution layer followed by Relu and MaxPooling layers

model.add(Flatten())
model.add(Dropout(0.5))
#Flatten layer to stack the output convolutions from second convolution layer
model.add(Dense(50,activation='relu'))
#Dense layer of 64 neurons
model.add(Dense(2,activation='softmax'))
#The Final layer with two outputs for two categories

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
checkpoint = ModelCheckpoint('best one',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')
history=model.fit(X_train,Y_train,epochs=2,callbacks=[checkpoint],validation_split=0.2)

In [ ]:
model.save(r"C:\Users\LEN\Downloads\Model.h5")

In [ ]:
from keras.models import load_model
import cv2
import numpy as np
from pygame import mixer
import os
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders

In [ ]:
mixer.init()
sound = mixer.Sound(r"C:\Users\LEN\Downloads\FaceMask_detection_alarm.wav")



model = load_model(r"C:\Users\LEN\Downloads\Model.h5")

face_clsfr = cv2.CascadeClassifier(r"C:\Users\LEN\Downloads\haarcascade_frontalface_default.xml")

In [ ]:
cap = cv2.VideoCapture(0)

labels_dict = {1: 'innocent', 0: 'intruder'}
color_dict = {0: (0, 0, 255), 1: (0, 255, 0)}


while (True):

    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_clsfr.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:

        face_img = gray[y:y + w, x:x + w]
        resized = cv2.resize(face_img, (100, 100))
        normalized = resized / 255.0
        reshaped = np.reshape(normalized, (1, 100, 100, 1))
        result = model.predict(reshaped)

        label = np.argmax(result, axis=1)[0]

        cv2.rectangle(frame, (x, y), (x + w, y + h), color_dict[label], 4)
        cv2.rectangle(frame, (x, y - 40), (x + w, y), color_dict[label], 4)
        cv2.putText(frame, labels_dict[label], (x, y - 10), cv2.FONT_ITALIC, 1, (255, 255, 255), 4)
        
        if(label==0):
            sound.play()
            crop_img = frame[y: y + h, x: x + w]
            cv2.imwrite(r"C:\Users\LEN\Downloads\face.jpg", crop_img)
            


            email_user='funnybro017@gmail.com'
            email_send='funnybro017@gmail.com'
            subject='Alert'

            msg=MIMEMultipart()
            msg['From']=email_user
            msg['To']=email_send
            msg['Subject']=subject

            body='Some one entered your house please be alert and please check the attachment  of the person'
            msg.attach(MIMEText(body,'plain'))

            filename=r"C:\Users\LEN\Downloads\face.jpg"
            attachment=open(filename,'rb')

            part=MIMEBase('application','octet-stream')
            part.set_payload((attachment).read())
            encoders.encode_base64(part)
            part.add_header('Content-Disposition',"attachment;filename= "+filename)

            msg.attach(part)
            text=msg.as_string()
            server=smtplib.SMTP('smtp.gmail.com',587)
            server.starttls()
            server.login(email_user,'9390148262')
            server.sendmail(email_user,email_send,text)
            server.quit()

        else:
            pass
            break

    cv2.imshow('Theft Detection ', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()
